# Introduction to Agents

####  Function calling

In [ ]:
!pip install ollama chromadb PyPDF2

In [ ]:
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import json
import re

In [ ]:
# Initialize Ollama
ollama = Ollama(
    base_url='http://localhost:11434',
    model="llama3.2"
)

# Simple function to call
def calculate_area(length, width):
    """Calculate area of a rectangle"""
    result = length * width
    return f"The area is {result} square units"

def get_weather(city):
    """Mock weather function"""
    weather_data = {
        "New York": "Sunny, 75°F",
        "London": "Rainy, 60°F", 
        "Cairo": "Hot, 95°F"
    }
    return weather_data.get(city, "Weather data not available")

# Function calling template
function_template = """
You can call these functions:
- calculate_area(length, width) - calculates rectangle area
- get_weather(city) - gets weather for a city

Question: {question}

If you need to use a function, respond with:
FUNCTION_CALL: function_name(param1, param2)

Otherwise, just answer normally.
"""

prompt = PromptTemplate(
    input_variables=["question"],
    template=function_template
)

chain = LLMChain(llm=ollama, prompt=prompt)

def process_response(question):
    response = chain.invoke({"question": question})
    answer = response['text']
    
    # Check if LLM wants to call a function
    if "FUNCTION_CALL:" in answer:
        # Extract function call
        match = re.search(r'FUNCTION_CALL:\s*(\w+)\((.*?)\)', answer)
        if match:
            func_name = match.group(1)
            params = match.group(2)
            
            print(f"LLM wants to call: {func_name}({params})")
            
            # Execute the function
            if func_name == "calculate_area":
                # Parse parameters
                nums = [float(x.strip()) for x in params.split(',')]
                result = calculate_area(nums[0], nums[1])
                
            elif func_name == "get_weather":
                city = params.strip().strip('"\'')
                result = get_weather(city)
            
            print(f"Function result: {result}")
            
            # Get final response with function result
            final_template = """
            Question: {question}
            Function result: {function_result}
            
            Please provide a complete answer using this function result.
            """
            
            final_prompt = PromptTemplate(
                input_variables=["question", "function_result"],
                template=final_template
            )
            
            final_chain = LLMChain(llm=ollama, prompt=final_prompt)
            final_response = final_chain.invoke({
                "question": question,
                "function_result": result
            })
            
            return final_response['text']
    
    return answer

# Test the function calling
print("=== Test 1: Area calculation ===")
result1 = process_response("What's the area of a rectangle that is 5 meters long and 3 meters wide?")
print("Final Answer:", result1)

print("\n=== Test 2: Weather query ===")
result2 = process_response("What's the weather like in Cairo?")
print("Final Answer:", result2)

print("\n=== Test 3: Regular question ===")
result3 = process_response("What causes northern lights?")
print("Final Answer:", result3)

=== Test 1: Area calculation ===
LLM wants to call: calculate_area(5, 3)
Function result: The area is 15.0 square units
Final Answer: To calculate the area of a rectangle, you can use the formula:

Area = length x width

In this case, the length of the rectangle is 5 meters and the width is 3 meters.

Area = 5 x 3
= 15.0 square units

So, the area of the rectangle is indeed 15.0 square units.

=== Test 2: Weather query ===
LLM wants to call: get_weather("Cairo")
Function result: Hot, 95°F
Final Answer: I don't have real-time access to current weather conditions. However, I can suggest some options to help you find out the current weather in Cairo.

You can check online weather websites such as AccuWeather, Weather.com, or the National Weather Service (NWS) for the most up-to-date information on weather conditions in Cairo.

Alternatively, you can also check social media or news websites for any updates on the weather in Cairo.

=== Test 3: Regular question ===
LLM wants to call: get_we

# Agentic Rag 

Just adding Rag functionality to Function Calling

In [ ]:
import ollama
import chromadb
import PyPDF2
import os
import re
import uuid
import json
import math
from datetime import datetime, timedelta

In [ ]:
def extract_pdf_text(pdf_path):
    """Extract text from a PDF file"""
    text = ""
    try:
        with open(pdf_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            for page in pdf_reader.pages:
                text += page.extract_text() + "\n"
    except:
        print(f"Could not read {pdf_path}")
    return text

# Setup documents
pdf_file = "your_document.pdf"
if os.path.exists(pdf_file):
    raw_text = extract_pdf_text(pdf_file)
    print(f"Extracted {len(raw_text)} characters from PDF")
else:
    # Sample text with some numbers and data for tool usage
    raw_text = """
    Machine learning algorithms require computational resources and data for training.
    A typical neural network might have 1000 parameters and require 100 hours of training time.
    Deep learning models can have millions of parameters, with training costs reaching $50,000 for large models.
    The accuracy of machine learning models typically ranges from 80% to 95% on standard datasets.
    Popular machine learning frameworks include TensorFlow (released in 2015), PyTorch (released in 2016), and Scikit-learn (released in 2007).
    Training a GPT-3 model cost approximately $4.6 million and required 175 billion parameters.
    A standard computer vision model might achieve 92% accuracy on ImageNet with 25 million parameters.
    The data preprocessing phase typically takes 60% of a machine learning project timeline.
    Cloud computing costs for ML training can range from $0.10 to $3.00 per hour depending on GPU type.
    """
    print("Using sample text with numbers for agent tool usage")

# Clean and chunk text
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s.,!?$%-]', '', text)
    return text.strip()

def split_into_chunks(text, chunk_size=150):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size):
        chunk = ' '.join(words[i:i + chunk_size])
        chunks.append(chunk)
    return chunks

clean_text_content = clean_text(raw_text)
text_chunks = split_into_chunks(clean_text_content)
print(f"Created {len(text_chunks)} chunks")

Using sample text with numbers for agent tool usage
Created 1 chunks


In [ ]:
def get_embedding(text):
    try:
        response = ollama.embeddings(model="nomic-embed-text", prompt=text)
        return response['embedding']
    except:
        print(f"Error getting embedding")
        return []

# Get embeddings
embeddings = []
for i, chunk in enumerate(text_chunks):
    print(f"Processing chunk {i+1}/{len(text_chunks)}")
    embedding = get_embedding(chunk)
    embeddings.append(embedding)

# Setup ChromaDB
client = chromadb.PersistentClient(path="./agent_rag_db")

try:
    client.delete_collection("agent_rag")
except:
    pass

collection = client.create_collection(name="agent_rag", metadata={"hnsw:space": "cosine"})

# Add documents
chunk_ids = [f"chunk_{i}" for i in range(len(text_chunks))]
chunk_metadata = [{"chunk_number": i, "source": "document"} for i in range(len(text_chunks))]

collection.add(
    documents=text_chunks,
    embeddings=embeddings,
    ids=chunk_ids,
    metadatas=chunk_metadata
)

print(f"Vector store ready with {len(text_chunks)} documents")

Processing chunk 1/1
Vector store ready with 1 documents


In [ ]:
# Tool 1: Calculator
def calculator(expression):
    """
    Calculate mathematical expressions. 
    Usage: calculator("2 + 2") or calculator("sqrt(16)") or calculator("log(100)")
    """
    try:
        # Replace common math functions
        expression = expression.replace("sqrt", "math.sqrt")
        expression = expression.replace("log", "math.log")
        expression = expression.replace("sin", "math.sin") 
        expression = expression.replace("cos", "math.cos")
        expression = expression.replace("^", "**")  # Power operator
        
        # Safe evaluation
        result = eval(expression, {"__builtins__": {}, "math": math})
        return f"Result: {result}"
    except Exception as e:
        return f"Calculator error: {str(e)}"

# Tool 2: Text Analyzer
def text_analyzer(text):
    """
    Analyze text statistics.
    Usage: text_analyzer("some text to analyze")
    """
    words = text.split()
    chars = len(text)
    sentences = len([s for s in text.split('.') if s.strip()])
    
    return f"""Text Analysis:
- Words: {len(words)}
- Characters: {chars}
- Sentences: {sentences}
- Average word length: {chars/len(words):.1f}
- Longest word: {max(words, key=len) if words else 'None'}"""

# Tool 3: Date Calculator
def date_calculator(operation, days=0):
    """
    Calculate dates. 
    Usage: date_calculator("add", 30) or date_calculator("subtract", 7)
    """
    today = datetime.now()
    
    if operation == "add":
        new_date = today + timedelta(days=days)
        return f"Today + {days} days = {new_date.strftime('%Y-%m-%d')}"
    elif operation == "subtract":
        new_date = today - timedelta(days=days)
        return f"Today - {days} days = {new_date.strftime('%Y-%m-%d')}"
    elif operation == "today":
        return f"Today is {today.strftime('%Y-%m-%d %H:%M')}"
    else:
        return "Usage: date_calculator('add'|'subtract'|'today', days)"

# Tool 4: Number Extractor
def number_extractor(text):
    """
    Extract all numbers from text.
    Usage: number_extractor("I have 5 apples and 3 oranges")
    """
    import re
    numbers = re.findall(r'-?\d+\.?\d*', text)
    if numbers:
        return f"Found numbers: {numbers}"
    else:
        return "No numbers found in text"

# Tool 5: Simple Counter
def counter(items_text):
    """
    Count items in comma-separated text.
    Usage: counter("apple, banana, orange, apple")
    """
    items = [item.strip().lower() for item in items_text.split(',')]
    count_dict = {}
    for item in items:
        count_dict[item] = count_dict.get(item, 0) + 1
    
    result = "Item counts:\n"
    for item, count in sorted(count_dict.items()):
        result += f"- {item}: {count}\n"
    return result

# Tool 6: Document Search (Our RAG retrieval)
def document_search(query):
    """
    Search through documents for relevant information.
    Usage: document_search("machine learning costs")
    """
    query_embedding = get_embedding(query)
    
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=3,
        include=['documents', 'distances', 'metadatas']
    )
    
    if results['documents'] and len(results['documents'][0]) > 0:
        found_docs = []
        for i, doc in enumerate(results['documents'][0]):
            distance = results['distances'][0][i]
            similarity = 1 - distance
            found_docs.append(f"Document {i+1} (relevance: {similarity:.2f}): {doc}")
        
        return "Found relevant documents:\n" + "\n\n".join(found_docs)
    else:
        return "No relevant documents found"

# Create tools registry
AVAILABLE_TOOLS = {
    "calculator": calculator,
    "text_analyzer": text_analyzer,
    "date_calculator": date_calculator, 
    "number_extractor": number_extractor,
    "counter": counter,
    "document_search": document_search
}

print("🛠️ Available Tools:")
for tool_name, tool_func in AVAILABLE_TOOLS.items():
    print(f"- {tool_name}: {tool_func.__doc__.strip().split('.')[0]}")

🛠️ Available Tools:
- calculator: Calculate mathematical expressions
- text_analyzer: Analyze text statistics
- date_calculator: Calculate dates
- number_extractor: Extract all numbers from text
- counter: Count items in comma-separated text
- document_search: Search through documents for relevant information


In [ ]:
# Test each tool
print("🧪 Testing Tools:")
print("=" * 40)

print("1. Calculator:")
print(calculator("2 + 2 * 3"))
print(calculator("sqrt(16)"))

print("\n2. Text Analyzer:")
print(text_analyzer("Hello world! This is a test."))

print("\n3. Date Calculator:")
print(date_calculator("today"))
print(date_calculator("add", 30))

print("\n4. Number Extractor:")
print(number_extractor("I spent $25.50 on 3 items"))

print("\n5. Counter:")
print(counter("apple, banana, apple, orange, apple"))

print("\n6. Document Search:")
print(document_search("machine learning training costs"))

Number of requested results 3 is greater than number of elements in index 1, updating n_results = 1


🧪 Testing Tools:
1. Calculator:
Result: 8
Result: 4.0

2. Text Analyzer:
Text Analysis:
- Words: 6
- Characters: 28
- Sentences: 1
- Average word length: 4.7
- Longest word: world!

3. Date Calculator:
Today is 2025-05-22 16:35
Today + 30 days = 2025-06-21

4. Number Extractor:
Found numbers: ['25.50', '3']

5. Counter:
Item counts:
- apple: 3
- banana: 1
- orange: 1


6. Document Search:
Found relevant documents:
Document 1 (relevance: 0.84): Machine learning algorithms require computational resources and data for training. A typical neural network might have 1000 parameters and require 100 hours of training time. Deep learning models can have millions of parameters, with training costs reaching $50,000 for large models. The accuracy of machine learning models typically ranges from 80% to 95% on standard datasets. Popular machine learning frameworks include TensorFlow released in 2015, PyTorch released in 2016, and Scikit-learn released in 2007. Training a GPT-3 model cost approximate

In [ ]:
def call_llm(prompt):
    """Call Ollama LLM"""
    try:
        response = ollama.chat(
            model="llama3.2:1b",
            messages=[{"role": "user", "content": prompt}]
        )
        return response['message']['content']
    except Exception as e:
        return f"LLM Error: {e}"

def parse_tool_call(text):
    """Parse tool calls from LLM response"""
    # Look for tool calls in format: TOOL_NAME(arguments)
    import re
    
    # Pattern to match: TOOL_NAME(arguments)
    pattern = r'(\w+)\((.*?)\)'
    matches = re.findall(pattern, text)
    
    tool_calls = []
    for tool_name, args in matches:
        if tool_name.lower() in AVAILABLE_TOOLS:
            tool_calls.append((tool_name.lower(), args))
    
    return tool_calls

def execute_tool(tool_name, args_string):
    """Execute a tool with parsed arguments"""
    try:
        tool_func = AVAILABLE_TOOLS[tool_name]
        
        # Parse arguments based on tool
        if tool_name == "calculator":
            return tool_func(args_string.strip('"\''))
        elif tool_name == "text_analyzer":
            return tool_func(args_string.strip('"\''))
        elif tool_name == "date_calculator":
            # Parse operation and days
            parts = [p.strip().strip('"\'') for p in args_string.split(',')]
            operation = parts[0] if parts else "today"
            days = int(parts[1]) if len(parts) > 1 and parts[1].isdigit() else 0
            return tool_func(operation, days)
        elif tool_name == "number_extractor":
            return tool_func(args_string.strip('"\''))
        elif tool_name == "counter":
            return tool_func(args_string.strip('"\''))
        elif tool_name == "document_search":
            return tool_func(args_string.strip('"\''))
        else:
            return f"Unknown tool: {tool_name}"
            
    except Exception as e:
        return f"Tool execution error: {str(e)}"

# Test tool calling
test_response = "I need to calculate 15 * 8 using calculator(15 * 8) and search for info using document_search(neural networks)"
tool_calls = parse_tool_call(test_response)
print(f"Parsed tool calls: {tool_calls}")

for tool_name, args in tool_calls:
    result = execute_tool(tool_name, args)
    print(f"{tool_name}({args}) -> {result}")

Number of requested results 3 is greater than number of elements in index 1, updating n_results = 1


Parsed tool calls: [('calculator', '15 * 8'), ('document_search', 'neural networks')]
calculator(15 * 8) -> Result: 120
document_search(neural networks) -> Found relevant documents:
Document 1 (relevance: 0.63): Machine learning algorithms require computational resources and data for training. A typical neural network might have 1000 parameters and require 100 hours of training time. Deep learning models can have millions of parameters, with training costs reaching $50,000 for large models. The accuracy of machine learning models typically ranges from 80% to 95% on standard datasets. Popular machine learning frameworks include TensorFlow released in 2015, PyTorch released in 2016, and Scikit-learn released in 2007. Training a GPT-3 model cost approximately $4.6 million and required 175 billion parameters. A standard computer vision model might achieve 92% accuracy on ImageNet with 25 million parameters. The data preprocessing phase typically takes 60% of a machine learning project time

In [ ]:
def create_agent_prompt(question):
    """Create prompt that teaches the LLM to use tools"""
    
    tools_description = """Available Tools:
- calculator(expression): Calculate math expressions like "2+2" or "sqrt(16)"
- text_analyzer(text): Analyze text statistics 
- date_calculator(operation, days): Calculate dates like date_calculator("add", 30)
- number_extractor(text): Extract numbers from text
- counter(items): Count comma-separated items
- document_search(query): Search documents for information

To use a tool, write: TOOL_NAME(arguments)
Example: calculator("2 + 2") or document_search("machine learning")
"""

    prompt = f"""{tools_description}

Question: {question}

Think step by step and use tools if needed to answer this question. If you need to use a tool, write the tool call clearly like: TOOL_NAME(arguments)

Your response:"""

    return prompt

def agentic_rag(question, max_iterations=3):
    """Agentic RAG that can use tools iteratively"""
    
    print(f"🤖 AGENTIC RAG")
    print(f"Question: {question}")
    print("=" * 60)
    
    conversation_history = []
    
    for iteration in range(max_iterations):
        print(f"\n🔄 Iteration {iteration + 1}")
        print("-" * 30)
        
        # Create prompt with history
        if iteration == 0:
            prompt = create_agent_prompt(question)
        else:
            # Add previous results to context
            history_text = "\n".join(conversation_history)
            prompt = f"""Previous conversation:
{history_text}

Continue working on the question: {question}
Use tools if you need more information or calculations.

Your response:"""
        
        # Get LLM response
        print("🧠 LLM thinking...")
        llm_response = call_llm(prompt)
        print(f"LLM Response: {llm_response}")
        
        # Parse and execute tool calls
        tool_calls = parse_tool_call(llm_response)
        
        if tool_calls:
            print(f"\n🛠️ Using {len(tool_calls)} tools:")
            tool_results = []
            
            for tool_name, args in tool_calls:
                print(f"Executing: {tool_name}({args})")
                result = execute_tool(tool_name, args)
                print(f"Result: {result}")
                tool_results.append(f"{tool_name}({args}) -> {result}")
            
            # Add to conversation history
            conversation_history.append(f"LLM: {llm_response}")
            conversation_history.append(f"Tool Results: " + "; ".join(tool_results))
            
        else:
            # No tools used, this is likely the final answer
            print(f"\n✅ Final Answer: {llm_response}")
            return llm_response
    
    print(f"\n⚠️ Reached max iterations ({max_iterations})")
    return llm_response

# Test the agentic RAG
agentic_rag("What is the cost of training machine learning models and how much would it cost to train for 200 hours?")

🤖 AGENTIC RAG
Question: What is the cost of training machine learning models and how much would it cost to train for 200 hours?

🔄 Iteration 1
------------------------------
🧠 LLM thinking...
LLM Response: TOOL_NAME(arguments)
calculation_tool("cost_of_training_machine_learning_model", "machine learning")
calculation_tool("training_cost_200_hours", 200)

✅ Final Answer: TOOL_NAME(arguments)
calculation_tool("cost_of_training_machine_learning_model", "machine learning")
calculation_tool("training_cost_200_hours", 200)


'TOOL_NAME(arguments)\ncalculation_tool("cost_of_training_machine_learning_model", "machine learning")\ncalculation_tool("training_cost_200_hours", 200)'

In [ ]:
# Test various scenarios that need different tools
# expected tools are only for reference, to compare if llm uses the right tools, not used in code
scenarios = [
    {
        "question": "How many parameters does a typical neural network have, and what's the square root of that number?",
        "expected_tools": ["document_search", "calculator"]
    },
    {
        "question": "If I start training a model today and it takes 100 hours, what date will it finish?",
        "expected_tools": ["date_calculator", "document_search"]
    },
    {
        "question": "Extract all the numbers from information about machine learning costs",
        "expected_tools": ["document_search", "number_extractor"]
    },
    {
        "question": "Analyze the text about machine learning and count how many times different frameworks are mentioned",
        "expected_tools": ["document_search", "text_analyzer"]
    }
]

print("🧪 Testing Different Agent Scenarios")
print("=" * 60)

for i, scenario in enumerate(scenarios):
    print(f"\n🔬 Test {i+1}: {scenario['question']}")
    print(f"Expected tools: {scenario['expected_tools']}")
    print("=" * 50)
    
    result = agentic_rag(scenario['question'], max_iterations=2)
    print("\n" + "🔄" * 20)

🧪 Testing Different Agent Scenarios

🔬 Test 1: How many parameters does a typical neural network have, and what's the square root of that number?
Expected tools: ['document_search', 'calculator']
🤖 AGENTIC RAG
Question: How many parameters does a typical neural network have, and what's the square root of that number?

🔄 Iteration 1
------------------------------
🧠 LLM thinking...
LLM Response: TOOL_NAME(arguments)
TOOL_NAME(arguments) 2
TOOL_NAME(arguments) sqrt(2)

✅ Final Answer: TOOL_NAME(arguments)
TOOL_NAME(arguments) 2
TOOL_NAME(arguments) sqrt(2)

🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄

🔬 Test 2: If I start training a model today and it takes 100 hours, what date will it finish?
Expected tools: ['date_calculator', 'document_search']
🤖 AGENTIC RAG
Question: If I start training a model today and it takes 100 hours, what date will it finish?

🔄 Iteration 1
------------------------------
🧠 LLM thinking...
LLM Response: TOOL_NAME(arguments) = calculator("add", 100)

To calculate how many days the mode

Number of requested results 3 is greater than number of elements in index 1, updating n_results = 1


LLM Response: I can help with that. To extract numbers from the given document, I'll use a combination of natural language processing (NLP) and regular expressions techniques.

First, let's analyze the document:

"Machine Learning Costs: The average cost per day for training a machine learning model ranges from $5,000 to $50,000 depending on the complexity of the task. However, for large-scale projects with multiple teams working simultaneously, costs can skyrocket up to $500,000 or more. Additionally, the time-consuming process of data preprocessing and feature engineering can extend hours beyond what was initially planned."

Now, let's use some tools to extract numbers from this text:

1. **Google's Ngram Viewer**: This tool can help us identify patterns in word frequencies over time. By analyzing the frequency of words like "machine learning", "costs", "average", and "range", we can infer that these are the main topics being discussed.
2. **TextBlob**: This API provides a set of mac

In [ ]:
def analyze_agent_decisions():
    """Analyze when the agent chooses to use different tools"""
    
    test_questions = [
        "What is 25 multiplied by 48?",  # Should use calculator
        "How many words are in the machine learning text?",  # Should use text_analyzer
        "What date is 45 days from today?",  # Should use date_calculator  
        "What are the costs mentioned in the documents?",  # Should use document_search + number_extractor
        "Tell me about neural networks"  # Should use document_search only
    ]
    
    print("🔍 Agent Decision Making Analysis")
    print("=" * 50)
    
    for question in test_questions:
        print(f"\n❓ Question: {question}")
        print("-" * 40)
        
        # Create prompt and get initial response
        prompt = create_agent_prompt(question)
        response = call_llm(prompt)
        
        # Analyze what tools the agent wants to use
        tool_calls = parse_tool_call(response)
        
        print(f"Agent's reasoning: {response[:150]}...")
        print(f"Tools chosen: {[call[0] for call in tool_calls]}")
        
        if tool_calls:
            print("Tool execution:")
            for tool_name, args in tool_calls:
                result = execute_tool(tool_name, args)
                print(f"  {tool_name}({args}) -> {result[:100]}...")
        else:
            print("No tools used - direct answer")
        
        print("=" * 50)

analyze_agent_decisions()

🔍 Agent Decision Making Analysis

❓ Question: What is 25 multiplied by 48?
----------------------------------------


Number of requested results 3 is greater than number of elements in index 1, updating n_results = 1
Number of requested results 3 is greater than number of elements in index 1, updating n_results = 1


Agent's reasoning: document_search("math multiplication")...
Tools chosen: ['document_search']
Tool execution:
  document_search("math multiplication") -> Found relevant documents:
Document 1 (relevance: 0.40): Machine learning algorithms require computat...

❓ Question: How many words are in the machine learning text?
----------------------------------------
Agent's reasoning: document_search("machine learning")...
Tools chosen: ['document_search']
Tool execution:
  document_search("machine learning") -> Found relevant documents:
Document 1 (relevance: 0.74): Machine learning algorithms require computat...

❓ Question: What date is 45 days from today?
----------------------------------------


Number of requested results 3 is greater than number of elements in index 1, updating n_results = 1


Agent's reasoning: date_calculator("subtract", 45)
 

or
TOOL_NAME(arguments): date_calculator("add", days=45)...
Tools chosen: ['date_calculator', 'date_calculator']
Tool execution:
  date_calculator("subtract", 45) -> Today - 45 days = 2025-04-07...
  date_calculator("add", days=45) -> Today + 0 days = 2025-05-22...

❓ Question: What are the costs mentioned in the documents?
----------------------------------------
Agent's reasoning: document_search("costs")...
Tools chosen: ['document_search']
Tool execution:
  document_search("costs") -> Found relevant documents:
Document 1 (relevance: 0.55): Machine learning algorithms require computat...

❓ Question: Tell me about neural networks
----------------------------------------


Number of requested results 3 is greater than number of elements in index 1, updating n_results = 1


Agent's reasoning: TOOL_NAME(arguments)
calculator("2 + 2") 
document_search("machine learning")
text_analyzer("neural networks")
counter(["artificial intelligence", "ma...
Tools chosen: ['calculator', 'document_search', 'text_analyzer', 'counter', 'date_calculator']
Tool execution:
  calculator("2 + 2") -> Result: 4...
  document_search("machine learning") -> Found relevant documents:
Document 1 (relevance: 0.74): Machine learning algorithms require computat...
  text_analyzer("neural networks") -> Text Analysis:
- Words: 2
- Characters: 15
- Sentences: 1
- Average word length: 7.5
- Longest word:...
  counter(["artificial intelligence", "machine learning"]) -> Item counts:
- "machine learning"]: 1
- ["artificial intelligence": 1
...
  date_calculator("add", 30) -> Today + 30 days = 2025-06-21...


# ReACT Agent

In [17]:
!pip install wikipedia -q --user


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


next cell performance is really dependent on LLM capabilities
if llama 3 used it will not be very efficient .. gemma works well

In [23]:
from langchain.agents import initialize_agent, Tool
from langchain.agents.agent_types import AgentType
from langchain_community.llms import Ollama
import requests
import wikipedia

# Initialize LLM
llm = Ollama(model="gemma2")

# Math tool using plain Python
def simple_calculator(expression: str) -> str:
    try:
        return str(eval(expression))
    except Exception as e:
        return f"Error: {e}"

calculator_tool = Tool(
    name="Calculator",
    func=simple_calculator,
    description="Useful for evaluating basic math expressions like '7.2 * 13'"
)

# wiki tool using Wikipedia API
def wiki_search(topic: str) -> str:
    try:
        return wikipedia.summary(topic, sentences=2)
    except Exception as e:
        return f"Error: {e}"

wiki_tool = Tool(
    name="Wikipedia",
    func=wiki_search,
    description="Useful for retrieving summaries of general knowledge topics from Wikipedia"
)

# Assemble tools and agent
tools = [calculator_tool, wiki_tool]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose=True
)

# Run query
query = "What is Alan Turing known for and what is 2 raised to the 8th power?"
response = agent.run(query)
print(response)




> Entering new AgentExecutor chain...
Thought: I need to find information about Alan Turing and calculate 2 to the 8th power. 
Action: Wikipedia
Action Input: Alan Turing
Observation: Error: Page id "alan tuning" does not match any pages. Try another id!
Thought:Thought: That's strange, let me try again with the correct spelling.  
Action: Wikipedia
Action Input: Alan Turing
Observation: Error: Page id "alan tuning" does not match any pages. Try another id!
Thought:Thought: Thought: Hmm, there seems to be an issue with my Wikipedia access. I'll have to calculate 2 raised to the 8th power using the calculator.  
Action: Calculator
Action Input: 2**8
Observation: 256
Thought:Thought: I now know that Alan Turing is a famous computer scientist and mathematician, and 2 to the 8th power is 256.
Final Answer: Alan Turing is known for his work in theoretical computer science and artificial intelligence.  2 raised to the 8th power is 256. 




> Finished chain.
Alan Turing is known for his wo

# Code Agent

### Code SQL Agent from smol agents

In [5]:
!pip install smolagents -q --user


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# cmd / restart kernel
!huggingface-cli login

^C


In [1]:
from sqlalchemy import (
    Column,
    Float,
    Integer,
    MetaData,
    String,
    Table,
    create_engine,
    insert,
    inspect,
    text,
)


engine = create_engine("sqlite:///:memory:")
metadata_obj = MetaData()

# create city SQL table
table_name = "receipts"
receipts = Table(
    table_name,
    metadata_obj,
    Column("receipt_id", Integer, primary_key=True),
    Column("customer_name", String(16), primary_key=True),
    Column("price", Float),
    Column("tip", Float),
)
metadata_obj.create_all(engine)

rows = [
    {"receipt_id": 1, "customer_name": "Alan Payne", "price": 12.06, "tip": 1.20},
    {"receipt_id": 2, "customer_name": "Alex Mason", "price": 23.86, "tip": 0.24},
    {"receipt_id": 3, "customer_name": "Woodrow Wilson", "price": 53.43, "tip": 5.43},
    {"receipt_id": 4, "customer_name": "Margaret James", "price": 21.11, "tip": 1.00},
]
for row in rows:
    stmt = insert(receipts).values(**row)
    with engine.begin() as connection:
        cursor = connection.execute(stmt)

inspector = inspect(engine)
columns_info = [(col["name"], col["type"]) for col in inspector.get_columns("receipts")]

table_description = "Columns:\n" + "\n".join([f"  - {name}: {col_type}" for name, col_type in columns_info])
print(table_description)

from smolagents import tool


@tool
def sql_engine(query: str) -> str:
    """
    Allows you to perform SQL queries on the table. Returns a string representation of the result.
    The table is named 'receipts'. Its description is as follows:
        Columns:
        - receipt_id: INTEGER
        - customer_name: VARCHAR(16)
        - price: FLOAT
        - tip: FLOAT

    Args:
        query: The query to perform. This should be correct SQL.
    """
    output = ""
    with engine.connect() as con:
        rows = con.execute(text(query))
        for row in rows:
            output += "\n" + str(row)
    return output


from smolagents import CodeAgent, InferenceClientModel


agent = CodeAgent(
    tools=[sql_engine],
    model=InferenceClientModel(model_id="meta-llama/Meta-Llama-3.1-8B-Instruct"),
)
agent.run("Can you give me the name of the client who got the most expensive receipt?")

Columns:
  - receipt_id: INTEGER
  - customer_name: VARCHAR(16)
  - price: FLOAT
  - tip: FLOAT


c:\Users\Amr osama abdellatif\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Can you give me the name of the client who got the most expensive receipt?                                      │
│                                                                                                                 │
╰─ InferenceClientModel - meta-llama/Meta-Llama-3.1-8B-Instruct ──────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  query = "SELECT customer_name FROM receipts ORDER BY price DESC LIMIT 1"                                         
  most_expensive_client = sql_engine(query)                                                                        
  print("The most expensive client is:", most_expensive_client)                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(most_expensive_client)                                                                              
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 
('Woodrow Wilson',)

[Step 2: Duration 3.43 seconds| Input tokens: 4,306 | Output tokens: 192]

"\n('Woodrow Wilson',)"

## Code Agent execution 
Executing code safely in a containerized docker env through code agent 

In [5]:
!pip install smolagents[docker] -q --user


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
from smolagents import CodeAgent, InferenceClientModel, WebSearchTool


model = InferenceClientModel()

# Docker executor example
with CodeAgent(tools=[WebSearchTool()], model=model, executor_type="docker") as agent:
    output = agent.run("How many seconds would it take for a leopard at full speed to run through Pont des Arts?")
print("Docker executor result:", output)

# E2B executor example
with CodeAgent(tools=[WebSearchTool()], model=model, executor_type="e2b") as agent:
    output = agent.run("How many seconds would it take for a leopard at full speed to run through Pont des Arts?")
print("E2B executor result:", output)

# WebAssembly executor example
with CodeAgent(tools=[], model=model, executor_type="wasm") as agent:
    output = agent.run("Calculate the square root of 125.")
print("Wasm executor result:", output)
# TODO: Support tools
# with CodeAgent(tools=[VisitWebpageTool()], model=model, executor_type="wasm") as agent:
#     output = agent.run("What is the content of the Wikipedia page at https://en.wikipedia.org/wiki/Intelligent_agent?")

Initializing executor, hold on...

Building Docker image jupyter-kernel...

Starting container on 127.0.0.1:8888...

Container status: created, waiting...

Stopping and removing container cc035a7090f0...

Container cleanup completed

RuntimeError: Failed to initialize Jupyter kernel: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))